In [4]:
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np

In [5]:
image_folder="D:/Datasets/YOLO_2_Dataset/test/images"
label_folder="D:/Datasets/YOLO_2_Dataset/test/labels"

test_images= [f for f in os.listdir(image_folder) if f.endswith((".jpeg", ".JPEG"))]
test_labels= [f for f in os.listdir(label_folder) if f.endswith((".txt"))]
print(f"Gefundene testbilder: {len(test_images)}")
print(f"Gefundene testlabels: {len(test_labels)}")
print(os.path.splitext(test_images[0])[0])
image_bbox_pairs = [
                    (os.path.join(image_folder, img), os.path.join(label_folder,img.replace(".jpeg",".txt").replace(".JPEG",".txt")))
                    for img in  test_images
                    if img.replace(".jpeg",".txt").replace(".JPEG",".txt") in test_labels
]
print(f"Gefundene Paare: {len(image_bbox_pairs)}")


Gefundene testbilder: 2700
Gefundene testlabels: 2700
1.3.6.1.4.1.14519.5.2.1.6655.2359.100048574870269820953143768196
Gefundene Paare: 2700


In [6]:
def load_ground_truth(file_path, image_shape):
    h,w = image_shape[:2]
    boxes=[]

    with open(file_path, "r") as f:
        for line in f.readlines():
            class_id,x_center, y_center, width, height = map(float, line.strip().split())

            x_min = int((x_center - width/2)*w)
            y_min = int((y_center - width/2)*h)
            x_max = int((x_center + width/2)*w)
            y_max = int((y_center + width/2)*h)
        
            boxes.append([x_min,y_min,x_max,y_max])
    return boxes

In [7]:
model=YOLO("YOLO2/1/weights/best.pt")
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [8]:
predictions={}
ground_truths={}

for image_path, bbox_path in image_bbox_pairs:
    image = Image.open(image_path)

    image_np = np.array(image)

    gt_boxes = load_ground_truth(bbox_path, image_np.shape)
    ground_truths[os.path.basename(image_path)] = gt_boxes

    results = model(image_np)

    boxes = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    predictions[os.path.basename(image_path)]= {
        "boxes": boxes,
        "scores": scores,
        "classes": classes
    }


0: 512x512 1 class1, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 347.8ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 class1s, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 51

In [9]:
def compute_iou(box1, box2):
    """
    Berechnet Intersection over Union (IoU) für zwei Bounding Boxes.
    Boxformat: [x_min, y_min, x_max, y_max]
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    # Flächen von Intersection und Union berechnen
    intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union = box1_area + box2_area - intersection
    
    return intersection / union if union > 0 else 0

def evaluate(predictions, ground_truths, iou_threshold=0.5, score_threshold=0.25):
    """
    Evaluierung der Vorhersagen im Vergleich zu den Ground Truths.
    predictions: Dictionary mit Vorhersagen
    ground_truths: Dictionary mit Ground Truths
    """
    tp, fp, fn, tn = 0, 0, 0, 0

    for image_id, gt_boxes in ground_truths.items():
        pred_boxes = predictions[image_id]['boxes']
        pred_scores = predictions[image_id]['scores']
        
        # Filter Vorhersagen basierend auf Score Threshold
        # pred_boxes = pred_boxes[pred_scores >= score_threshold]
        
        # Fall: Keine Ground Truth und keine Vorhersagen
        if len(gt_boxes) == 0 and len(pred_boxes) == 0:
            tn += 1
            continue

        matched_gt = set()
        for pred_box in pred_boxes:
            # Berechne IoU mit allen Ground Truths
            ious = [compute_iou(pred_box, gt_box) for gt_box in gt_boxes]
            max_iou = max(ious) if ious else 0  # maximaler IoU-Wert für diese Vorhersage
            
            if max_iou >= iou_threshold:
                # True Positive: Eine Ground Truth wurde erkannt
                matched_gt.add(ious.index(max_iou))
            for iou in ious:
                if iou >= iou_threshold:
                    # True Positive: Eine Ground Truth wurde erkann
                    tp += 1
                else:
                    # False Positive: Keine passende Ground Truth gefunden
                    fp += 1

        
        # False Negatives: Nicht erkannte Ground Truths
        fn += len(gt_boxes) - len(matched_gt)

    # Metriken berechnen
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "specificity": specificity,
        "f1_score": f1_score,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "tn": tn,
    }


results = evaluate(predictions, ground_truths)
print(results)


{'precision': 0.8876204067070995, 'recall': 0.9014492753623189, 'accuracy': 0.8091056910569105, 'specificity': 0.0, 'f1_score': 0.8944813949307927, 'tp': 2488, 'fp': 315, 'fn': 272, 'tn': 0}
